In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing, tree
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV,cross_val_score

In [2]:
train = r"C:\Users\宫城楠木\Desktop\机器学习\实验三\泰坦尼克\train.csv"
test = r"C:\Users\宫城楠木\Desktop\机器学习\实验三\泰坦尼克\test.csv"
train_data = pd.read_csv(train)#导入训练数据
test_data= pd.read_csv(test)#导入测试数据
#导入预测的数据
fact_data = pd.read_csv(r'C:\Users\宫城楠木\Desktop\机器学习\实验三\gender_submission.csv')
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [6]:
#因为Cabin缺失太多了，姓名与船票号跟结果并无关系，基于
train_data.drop(["Cabin","Ticket","Name"],inplace = True,axis=1)
test_data.drop(["Cabin","Ticket","Name"],inplace = True,axis=1)
train_data['Embarked'].fillna(train_data['Embarked'].mode()[0], inplace = True)
train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [7]:
labels = train_data["Embarked"].unique().tolist()
labels 

['S', 'C', 'Q']

In [8]:
train_data["Age"] = train_data["Age"].fillna(train_data["Age"].mean())
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(2)
memory usage: 62.8+ KB


In [9]:
test_data["Age"] = test_data["Age"].fillna(test_data["Age"].mean())
test_data["Fare"] = test_data["Fare"].fillna(test_data["Fare"].mean())
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Sex          418 non-null    object 
 3   Age          418 non-null    float64
 4   SibSp        418 non-null    int64  
 5   Parch        418 non-null    int64  
 6   Fare         418 non-null    float64
 7   Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 26.2+ KB


In [10]:
train_data["Sex"] = (train_data["Sex"] == "male").astype("int")
train_data["Embarked"] = train_data["Embarked"].apply(lambda x:labels.index(x))
train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,0
1,2,1,1,0,38.0,1,0,71.2833,1
2,3,1,3,0,26.0,0,0,7.9250,0
3,4,1,1,0,35.0,1,0,53.1000,0
4,5,0,3,1,35.0,0,0,8.0500,0


In [11]:
test_data["Sex"] = (test_data["Sex"] == "male").astype("int")
test_data["Embarked"] = test_data["Embarked"].apply(lambda x:labels.index(x))
test_data.head()
test_data.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [12]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
train_features = train_data[features]
train_labels = train_data['Survived']

In [13]:
test_data

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.50000,0,0,7.8292,2
1,893,3,0,47.00000,1,0,7.0000,0
2,894,2,1,62.00000,0,0,9.6875,2
3,895,3,1,27.00000,0,0,8.6625,0
4,896,3,0,22.00000,1,1,12.2875,0
...,...,...,...,...,...,...,...,...
413,1305,3,1,30.27259,0,0,8.0500,0
414,1306,1,0,39.00000,0,0,108.9000,1
415,1307,3,1,38.50000,0,0,7.2500,0
416,1308,3,1,30.27259,0,0,8.0500,0


In [14]:
parameters = {
	    "min_samples_leaf":[*range(1,50,5)],
	    "min_impurity_decrease":[*np.linspace(0,0.5,50)]
	}
	
clf = DecisionTreeClassifier(random_state=25,max_depth=3)
GS = GridSearchCV(clf,parameters,cv=10)
GS = GS.fit(train_features,train_labels)

In [15]:
GS.best_score_

0.8170536828963794

In [16]:
GS.best_params_

{'min_impurity_decrease': 0.0, 'min_samples_leaf': 6}

In [17]:
clf = DecisionTreeClassifier(random_state=25,max_depth=3,criterion="entropy",min_samples_leaf=16,min_impurity_decrease=0)
clf = clf.fit(train_features,train_labels)
test_features = test_data[features]
pred_labels = clf.predict(test_features)
test_data.insert(1, 'Survived', pred_labels, allow_duplicates=False)
test_labels =test_data['Survived']
fact_labels = fact_data['Survived']

In [18]:
test_data

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,0,3,1,34.50000,0,0,7.8292,2
1,893,1,3,0,47.00000,1,0,7.0000,0
2,894,0,2,1,62.00000,0,0,9.6875,2
3,895,0,3,1,27.00000,0,0,8.6625,0
4,896,1,3,0,22.00000,1,1,12.2875,0
...,...,...,...,...,...,...,...,...,...
413,1305,0,3,1,30.27259,0,0,8.0500,0
414,1306,1,1,0,39.00000,0,0,108.9000,1
415,1307,0,3,1,38.50000,0,0,7.2500,0
416,1308,0,3,1,30.27259,0,0,8.0500,0


In [19]:
clf = clf.fit(train_features,train_labels)
score = clf.score(test_features,pred_labels)
score

1.0

In [20]:
with open('Titanic.dot', 'w') as f:
    dot_data = tree.export_graphviz(clf, out_file=None)
    f.write(dot_data)

In [21]:
print(u'cross_val_score准确率为 %.4lf' % np.mean(cross_val_score(clf, train_features, train_labels, cv=10)))

cross_val_score准确率为 0.8059


In [22]:
test_data[['PassengerId','Survived']].to_csv(r'C:\Users\宫城楠木\Desktop\机器学习\实验三\result.csv')

In [23]:
'''
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print("准确率:", accuracy_score(fact_labels, pred_labels))
print("精确度:", precision_score(fact_labels,pred_labels,average='micro'))
print("召回率:", recall_score(fact_labels, pred_labels,average='micro'))
print("f1   :", f1_score(fact_labels,pred_labels,average='micro'))
'''

'\nfrom sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score\nprint("准确率:", accuracy_score(fact_labels, pred_labels))\nprint("精确度:", precision_score(fact_labels,pred_labels,average=\'micro\'))\nprint("召回率:", recall_score(fact_labels, pred_labels,average=\'micro\'))\nprint("f1   :", f1_score(fact_labels,pred_labels,average=\'micro\'))\n'